In [ ]:
# %% Deep learning - Section 10.88
#    Code challenge 9: batch-normalise the qwerties

#    1) Use the data and the model from
#    2) Implement batch normalisation after the input layer
#    3) Optional: use a class

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

nClust = 100
blur   = 1

A = [1, 3]
B = [1,-2]

a = [ A[0]+np.random.randn(nClust)*blur, A[1]+np.random.randn(nClust)*blur ]
b = [ B[0]+np.random.randn(nClust)*blur, B[1]+np.random.randn(nClust)*blur ]

# True labels
labels_np = np.vstack(( np.zeros((nClust,1)), np.ones((nClust,1)) ))

# Concatenate
data_np = np.hstack((a,b)).T

# Convert into torch tensor
data   = torch.tensor(data_np).float()
labels = torch.tensor(labels_np).float()


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig = plt.figure(figsize=(6*phi,6))

plt.plot( data[np.where(labels==0)[0],0],data[np.where(labels==0)[0],1],'bs' )
plt.plot( data[np.where(labels==1)[0],0],data[np.where(labels==1)[0],1],'ro' )
plt.title('Some binary data')
plt.xlabel('x1')
plt.ylabel('x2')

plt.savefig('figure22_code_challenge_9.png')

plt.show()

files.download('figure22_code_challenge_9.png')


In [ ]:
# %% Model class

class ANN(nn.Module):
    def __init__(self,doBN=True):
        super().__init__()

        # Batch normalisation property
        self.doBN = doBN

        # Architecture and batch norm
        self.input  = nn.Linear( 2,16)
        self.hidden = nn.Linear(16,1 )
        self.bn1    = nn.BatchNorm1d(16)
        self.output = nn.Linear( 1,1 )

    # Forward propagation
    def forward(self,x):

        # Input (already normalised if data normalised)
        x = F.relu( self.input(x) )

        # Hidden layers (batchnorm, weighted sum, activation function)
        if self.doBN:

            x = self.bn1(x)
            x = self.hidden(x)
            x = F.relu(x)

        else:

            x = self.hidden( F.relu(x) )

        # Output
        x = torch.sigmoid( self.output(x) )

        return x


In [ ]:
# %% Function to train the model

num_epochs = 1000

def train_model(ANN,learning_rate):

    # Loss function and optimizer
    loss_fun = nn.BCELoss()
    optimizer = torch.optim.SGD(ANN.parameters(),lr=learning_rate)

    # Initialise losses
    losses = torch.zeros(num_epochs)

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Forward propagation
        yHat = ANN(data)

        # Compute loss
        loss = loss_fun(yHat,labels)
        losses[epoch_i] = loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Final forward pass
    predictions = ANN(data)

    # Accuracy (here .5 because manual Sigmoid implementation [no BCEWithLogitsLoss])
    tot_acc = 100*torch.mean( ((predictions>.5)==labels).float() )

    return losses,predictions,tot_acc


In [ ]:
# %% Test model once

model = ANN(doBN=True)
losses,predictions,tot_acc = train_model(model,0.01)

print(f'Final accuracy = {tot_acc}%')


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig = plt.figure(figsize=(6*phi,6))

plt.plot(losses.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model losses')

plt.savefig('figure23_code_challenge_9.png')

plt.show()

files.download('figure23_code_challenge_9.png')


In [ ]:
# %% More plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig = plt.figure(figsize=(6*phi,6))

plt.plot(predictions.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Data point')
plt.ylabel('Probability')
plt.title('Model predictions')

plt.savefig('figure24_code_challenge_9.png')

plt.show()

files.download('figure24_code_challenge_9.png')


In [ ]:
# %% Parametric experiment on learning rates

# Parameters
learning_rates   = np.linspace(.001,.1,50)
acc_by_learnRate = []
all_losses       = np.zeros( (len(learning_rates),num_epochs) )

# Loop through learning rates
for i,lr in enumerate(learning_rates):

    # Generate and run the model
    model = ANN(doBN=True)
    losses,predictions,tot_acc = train_model(model,lr)

    # Store results
    acc_by_learnRate.append(tot_acc)
    all_losses[i,:] = losses.detach().numpy()


In [ ]:
# %% Plotting

phi = ( 1 + np.sqrt(5) ) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*6*phi,6))

ax[0].plot(learning_rates,acc_by_learnRate,'s-')
ax[0].set_xlabel('Learning rate')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Accuracy by learning rate')

cmaps = plt.cm.plasma(np.linspace(.1,.9,len(all_losses.T[1])))
for i in range(len(all_losses.T[1])):
    ax[1].plot(all_losses.T[:,i],color=cmaps[i])

#ax[1].plot(all_losses.T)
ax[1].set_title('Losses by learning rate')
ax[1].set_xlabel('Epoch number')
ax[1].set_ylabel('Loss')

plt.savefig('figure25_code_challenge_9.png')

plt.show()

files.download('figure25_code_challenge_9.png')
